# Demonstrate Seq2Seq Wrapper with twitter chat log

In [9]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [10]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [11]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 512

In [12]:
import seq2seq_wrapper

In [13]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from 'C:\\Users\\ricsi\\Documents\\GitHub\\Python-projects\\seq2seq_chatbot_projects\\practical_seq2seq-master\\seq2seq_wrapper.py'>

In [14]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [15]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 128)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 128)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #1000
val   loss : 2.029175

Model saved to disk at iteration #2000
val   loss : 2.020699

Model saved to disk at iteration #3000
val   loss : 1.976326

Model saved to disk at iteration #4000
val   loss : 2.016675


In [39]:
sess = model.restore_last_session()

In [42]:
input_ = test_batch_gen.__next__()[0]
print(input_.shape)
output = model.predict(sess, input_)
print(output.shape)

(20, 256)
(256, 20)


In [41]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [this is what alicia machado looked like when trump took to publicly amp unk unk her over her weight sh]; a : [ she has a chance]
q : [i am jim amp my now then guy best friend is unk my heart hurts]; a : [is this the best thing ive seen it]
q : [anytime you want to come down and watch a game with the unk you are more than welcome]; a : [oh no i dont know what to do]
q : [we love a good cup of coffee with milk were excited for a without any thanks to ]; a : [we need to be in the bay area]
q : [you guys is so good ]; a : [youre so cute ]
q : [what year is considered the last of unk then ]; a : [the whole time is a good thing to the game]
q : [im happy my friends support my dogs insta]; a : [you are so cute]
q : [user survey done right by  unk short and easy low unk]; a : [so much the way to the end of the world is a great place]
q : [why did i tell my mom to update her phone to ios10 lmao now she wont leave me alone]; a : [lmao i was thinking about that lol i was just thinking about 